# Homework 4 - Computing Point-in-Time Residual Returns
In this homework, we will use regressions to compute beta-adjusted "residual" returns in a point-in-time fashion suitable for backtesting / live trading.


1. Download Daily Bars for FB, AAPL, AMZN, NFLX, GOOGL and QQQ from yahoo finance starting 2016-01-01. Use the Adj Close to compute daily returns.
2. Now, let's compute the beta of FB, AAPL, AMZN, NFLX, GOOGL using QQQ as our benchmark. You can think of this as the beta these stocks have to their industry (tech). In practice,  we have to use some lookback window to compute the beta. Let's use 252 (1 year, excluding wknds/holidays). So, for each day, the betas should be computed using the most recent 252 data points.
3. Using the betas, compute an "alpha" on each day. This is also known as a "residual return".
4. Compare the volatility of the residual returns to that of the original returns. What do you notice?
5. Compare the pairwise correlations of the residual returns to that of the original returns. What do you notice?
6. Compute the information ratio for each of these stocks and compare that to the sharpe ratio.


In [16]:
import yfinance as yf
import numpy as np


univ = ['META', 'AAPL', 'AMZN', 'NFLX', 'GOOGL', 'QQQ']
# Download historical data for the universe of stocks
data = yf.download(univ, start='2020-01-01', auto_adjust=False)['Adj Close']

# Get returns for each stock
returns = data.pct_change().dropna()

corr = returns.rolling(252).corr(returns['QQQ'])
vol = returns.rolling(252).std()
beta = (corr*vol).divide(vol['QQQ'],axis=0)  
beta

[*********************100%***********************]  6 of 6 completed


Ticker,AAPL,AMZN,GOOGL,META,NFLX,QQQ
Date,,,,,,
2020-01-03,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-06,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-07,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-08,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-09,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
2025-07-22,0.923696,1.111020,0.868760,1.131661,0.798682,1.0
2025-07-23,0.924862,1.111876,0.869369,1.133723,0.796152,1.0
2025-07-24,0.928061,1.116643,0.867352,1.132698,0.794251,1.0


In [17]:
alpha = returns - beta.multiply(returns['QQQ'], 0)
alpha

Ticker,AAPL,AMZN,GOOGL,META,NFLX,QQQ
Date,,,,,,
2020-01-03,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-06,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-07,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-08,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-09,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
2025-07-22,0.013817,-0.002230,0.011019,-0.005588,-0.030887,-6.938894e-18
2025-07-23,-0.005385,-0.001467,-0.009767,0.007272,-0.014807,6.938894e-18
2025-07-24,-0.003796,0.014882,0.008352,-0.000701,0.001692,3.469447e-18


In [18]:
import pandas as pd
vol = {}
vol['Original'] = returns.std()* np.sqrt(252)
vol['Residual Return'] = alpha.std() * np.sqrt(252)
df = pd.DataFrame(vol).drop('QQQ')
df

,Original,Residual Return
Ticker,,
AAPL,0.324244,0.170506
AMZN,0.359637,0.217047
GOOGL,0.326517,0.203956
META,0.444968,0.311853
NFLX,0.446788,0.356441


In [19]:
returns.corr()

Ticker,AAPL,AMZN,GOOGL,META,NFLX,QQQ
Ticker,,,,,,
AAPL,1.000000,0.601293,0.636149,0.567802,0.446663,0.832770
AMZN,0.601293,1.000000,0.653725,0.629902,0.532988,0.776944
GOOGL,0.636149,0.653725,1.000000,0.638527,0.437339,0.788329
META,0.567802,0.629902,0.638527,1.000000,0.495791,0.720542
NFLX,0.446663,0.532988,0.437339,0.495791,1.000000,0.592691
QQQ,0.832770,0.776944,0.788329,0.720542,0.592691,1.000000


In [20]:
alpha.corr()

Ticker,AAPL,AMZN,GOOGL,META,NFLX,QQQ
Ticker,,,,,,
AAPL,1.000000,-0.122899,-0.016119,-0.110311,-0.105747,-0.001720
AMZN,-0.122899,1.000000,0.131565,0.141332,0.067637,-0.025745
GOOGL,-0.016119,0.131565,1.000000,0.135044,-0.065179,-0.036934
META,-0.110311,0.141332,0.135044,1.000000,0.108290,-0.025063
NFLX,-0.105747,0.067637,-0.065179,0.108290,1.000000,-0.034840
QQQ,-0.001720,-0.025745,-0.036934,-0.025063,-0.034840,1.000000


In [21]:
df = {}
df['IR'] = alpha.mean() / alpha.std()*np.sqrt(252)
df['SR'] = returns.mean() / returns.std()*np.sqrt(252)
df = pd.DataFrame(df).drop('QQQ')
df

,IR,SR
Ticker,,
AAPL,-0.133762,0.762715
AMZN,-0.318008,0.629366
GOOGL,0.260858,0.738031
META,0.296127,0.725425
NFLX,0.307545,0.743516


##### SR is much greater because of the added beta component